Set FLAGS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
class FLAGS(object):
    """ """
    """
    General settings
    """
    input_size = (1024, 1024)
    heatmap_size = 128
    cpm_stages = 4
    joint_gaussian_variance = 1.0
    center_radius = 21
    num_of_joints = 8
    color_channel = 'RGB'
    normalize = True
    use_gpu = True
    gpu_id = 0
    
    gradient_clipping = True # gradient clipping

    """
    Demo settings
    """
    # 'MULTI': show multiple stage heatmaps
    # 'SINGLE': show last stage heatmap
    # 'Joint_HM': show last stage heatmap for each joint
    # 'image or video path': show detection on single image or video
    DEMO_TYPE = 'SINGLE'

    model_path = 'cpm_hand'
    cam_id = 0

    use_kalman = True
    kalman_noise = 0.03
    keypoints_order = ["TAIL_NOTCH",
                        "ADIPOSE_FIN",
                        "UPPER_LIP",
                        "ANAL_FIN",
                        "PELVIC_FIN",
                        "EYE",
                        "PECTORAL_FIN",
                        "DORSAL_FIN"]

    """
    Training settings
    """
    network_def = 'fish_test'
    train_img_dir = ''
    val_img_dir = ''
    bg_img_dir = ''
    pretrained_model = 'fish_test'
    batch_size = 4
    init_lr = 0.001
    lr_decay_rate = 0.45
    lr_decay_step = 8000
    augmentation = None
    buffer_range = [int(n) for n in np.arange(100, 600, 100)] # useless if crop = False
    crop = False # crop input image based on keypoints - for GTSF only
#     augmentation = albu.Compose([albu.HorizontalFlip(p=0.5),
# #                                  albu.Rotate(limit=10, p=1.0)
#                                 ], 
#                                  p=1.0,
#                                  keypoint_params={'format': 'xy'})
    
    epochs=200

    hnm = True  # Make sure generate hnm files first
    do_cropping = True

    """
    For Freeze graphs
    """
    output_node_names = 'stage_3/mid_conv7/BiasAdd:0'

Load generator

In [ ]:
from utils.utils import DataGenerator, data_generator
import time
import json

In [ ]:
annotations = json.load(open('/root/data/bati/labels/labels_2019-05-10.json'))

In [ ]:
for ann in annotations:
    ann['local_path'] = ann['local_path'].replace('/app', '/root')

In [ ]:
gen = data_generator(annotations, FLAGS)

In [ ]:
start = time.time()
for _ in range(100):
    img, kp, hm = next(gen)
end = time.time()
print("Loading time: {}".format(end - start))

In [ ]:
# @background(max_prefetch=10)
# def data_generator_async(annotations, FLAGS, nsteps=1000):
#     index = 0
#     while True:
#         xbatch = []
#         ybatch = []
#         start = index*FLAGS.batch_size
#         for i in range(FLAGS.batch_size):
#             buffer = np.random.choice(FLAGS.buffer_range)
#             image, keypoints = load_image_keypoints(annotations[start + i], 
#                                                     FLAGS,
#                                                     reshape=True,
#                                                     buffer=buffer)
#             xbatch.append(image)
#             ybatch.append(keypoints)
#         xbatch = np.array(xbatch)
#         if FLAGS.normalize:
#             xbatch = xbatch / 255.0 - 0.5
#         else:
#             xbatch -= 128.0
#         ybatch = np.array(ybatch)
#         batch_gt_heatmap_np = make_heatmaps_from_joints(FLAGS.input_size[0],
#                                                         FLAGS.heatmap_size,
#                                                         FLAGS.joint_gaussian_variance,
#                                                         ybatch)
#         index += 1
#         if index >= nsteps:
#             index = 0
#             random.shuffle(annotations)
#         yield xbatch, ybatch, batch_gt_heatmap_np

In [ ]:
# async_gen = data_generator_async(annotations, FLAGS)

In [ ]:
# start = time.time()
# for _ in range(100):
#     img, kp, hm = next(async_gen)
# end = time.time()
# print("Loading time: {}".format(end - start))

In [ ]:
from multiprocessing import Pool

In [ ]:
from utils.utils import load_image_keypoints, make_heatmaps_from_joints
import keras
import time

In [ ]:
# start = time.time()
# xbatch = []
# ybatch = []
# for i in range(4):
#     buffer = np.random.choice(FLAGS.buffer_range)
#     image, keypoints = load_image_keypoints(annotations[i], 
#                                             FLAGS,
#                                             reshape=True,
#                                             buffer=buffer)
#     xbatch.append(image)
#     ybatch.append(keypoints)   
# end = time.time()
# print("Loading time: {}".format(end - start))

In [ ]:
# start = time.time()
# pool = Pool(4)
# pool.apply()
# end = time.time()
# print("Loading time: {}".format(end - start))

In [ ]:
# train_generator = DataGenerator(annotations, FLAGS)
# start = time.time()
# for i in range(100):
#     output = train_generator[i]
# end = time.time()
# print("Loading time: {}".format((end - start)/100.0))

In [ ]:
class DataGenerator_async(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, annotations_set, FLAGS):
        'Initialization'
        self.annotations = annotations_set
        self.on_epoch_end()
        self.FLAGS = FLAGS
        self.batches = []
        self._preload_batches()
        
    def _log_result(self, result):
        self.batches.append(result)
        
    def _load_batch(self, anns):
        xbatch = []
        ybatch = []
        for ann in anns:
            buffer = np.random.choice(self.FLAGS.buffer_range)
            image, keypoints = load_image_keypoints(ann, 
                                                    self.FLAGS,
                                                    reshape=True,
                                                    buffer=buffer)
            xbatch.append(image)
            ybatch.append(keypoints)
        
        xbatch = np.array(xbatch)
        if self.FLAGS.normalize:
            xbatch = xbatch / 255.0 - 0.5
        else:
            xbatch -= 128.0
        ybatch = np.array(ybatch)
        batch_gt_heatmap_np = make_heatmaps_from_joints(self.FLAGS.input_size[0],
                                                        self.FLAGS.heatmap_size,
                                                        self.FLAGS.joint_gaussian_variance,
                                                        ybatch)   
        return xbatch, ybatch, batch_gt_heatmap_np
            
    
    def _preload_batches(self):
#         num_batches = int(math.ceil(len(self.annotations) / 4))
        num_batches = 1000
        pool = Pool(12)
        for i in range(num_batches):
            pool.apply_async(self._load_batch, 
                             args = (self.annotations[i*4: (i+1)*4],)
                             , callback = self._log_result)
        pool.close()
           
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.annotations) // self.FLAGS.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        return self.batches[index]

In [ ]:
start = time.time()
train_generator_async = DataGenerator_async(annotations, FLAGS)
end = time.time()
print("Loading time: {}".format(end - start))

In [ ]:
len(train_generator_async.batches)

In [ ]:
for i in range(50):
    start = time.time()
    img, kp, hm = train_generator_async[i]
    train_generator_async._preload_batches()
    time.sleep(0.5)
    end = time.time()
    print("Loading time: {}".format(end - start))
    
    
    print(img.shape, kp.shape, hm.shape)

# MODIFY

In [ ]:
from utils.cpm_utils import make_gaussian

In [ ]:
def make_heatmaps_from_joints(input_size, heatmap_size, gaussian_variance, batch_joints):
    # Generate ground-truth heatmaps from ground-truth 2d joints
    scale_factor = input_size // heatmap_size
    batch_gt_heatmap_np = []
    for i in range(batch_joints.shape[0]):
        gt_heatmap_np = []
        invert_heatmap_np = np.ones(shape=(heatmap_size, heatmap_size))
        for j in range(batch_joints.shape[1]):
            cur_joint_heatmap = make_gaussian(heatmap_size,
                                              gaussian_variance,
                                              center=(batch_joints[i][j] // scale_factor))
            gt_heatmap_np.append(cur_joint_heatmap)
            invert_heatmap_np -= cur_joint_heatmap
        gt_heatmap_np.append(invert_heatmap_np)
        batch_gt_heatmap_np.append(gt_heatmap_np)
    batch_gt_heatmap_np = np.asarray(batch_gt_heatmap_np)
    print(batch_gt_heatmap_np.shape)
    batch_gt_heatmap_np = np.transpose(batch_gt_heatmap_np, (0, 2, 3, 1))
    
    return batch_gt_heatmap_np

In [ ]:
def make_heatmap_single(input_size, heatmap_size, gaussian_variance, batch_joints):
    scale_factor = input_size // heatmap_size
    gt_heatmap_np = []
    invert_heatmap_np = np.ones(shape=(heatmap_size, heatmap_size))
    for j in range(batch_joints.shape[0]):
        cur_joint_heatmap = make_gaussian(heatmap_size,
                                          gaussian_variance,
                                          center=(batch_joints[j] // scale_factor))
        gt_heatmap_np.append(cur_joint_heatmap)
        invert_heatmap_np -= cur_joint_heatmap
    gt_heatmap_np.append(invert_heatmap_np)
    gt_heatmap_np = np.array(gt_heatmap_np)
    gt_heatmap_np = np.transpose(gt_heatmap_np, (1, 2, 0))
    return gt_heatmap_np

In [ ]:
def load_image_keypoints_gaussian(annotation):
    image, keypoints = load_image_keypoints(annotation, FLAGS)
#     print(keypoints.shape)
    heatmap = make_heatmap_single(FLAGS.input_size[0], 
                                FLAGS.heatmap_size, 
                                FLAGS.joint_gaussian_variance,
                                keypoints)
    return image, keypoints, heatmap

In [ ]:
img, kps, hm = load_image_keypoints_gaussian(annotations[0])

In [ ]:
start = time.time()
pool = Pool(8)
data = pool.map_async(load_image_keypoints_gaussian, annotations[:10000])
pool.close()
pool.join()
end = time.time()
print(end-start)

# ASYNC

In [ ]:
class DataGenerator_async(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, annotations_set, FLAGS):
        'Initialization'
        self.annotations = annotations_set
        self.on_epoch_end()
        self.FLAGS = FLAGS
        self.batch = []
        
        self.images = []
        self.keypoints = []
        self.heatmaps = []
        
        # get the first batch
        self._prefetch(0)
        
    def _log_result(self, result):
        self.images.append(result[0])
        self.keypoints.append(result[1])
        self.heatmaps.append(result[2])
        
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.annotations) // self.FLAGS.batch_size

    
    def _prefetch(self, index):
        print('Fetching')      
        
        pool = Pool(8)
        pool.map(load_image_keypoints_gaussian, 
                 args = (self.annotations[:125],)
                 , callback = self._log_result)
        pool.close()       
#         pool.join()      
    
    
    def __getitem__(self, index):
        'Generate one batch of data'
#         self._prefetch(index+1)
        
        images = np.array(self.images[:4])
        keypoints = np.array(self.keypoints[:4])
        heatmaps = np.array(self.heatmaps[:4])
        
        return images, keypoints, heatmaps

In [ ]:
pool = Pool(8)
pool.apply_async(load_image_keypoints_gaussian, 
                     args = (annotations[:125]))
pool.close()       

# TEST

In [ ]:
#         num_batches = 1000
#         pool = Pool(12)
#         for i in range(num_batches):
#             pool.apply_async(self._load_batch, 
#                              args = (self.annotations[i*4: (i+1)*4],)
#                              , callback = self._log_result)
#         pool.close()

In [ ]:
start = time.time()
train_generator_async = DataGenerator_async(annotations, FLAGS)
end = time.time()
print("Loading time: {}".format(end - start))

In [ ]:
len(train_generator_async.images)

In [ ]:
import time

In [ ]:
start = time.time()
img, kp, hm = train_generator_async[0]
end = time.time()
print("Loading time: {}".format(end - start))

In [ ]:
for i in range(50):
    start = time.time()
    img, kp, hm = train_generator_async[i]
    train_generator_async._prefetch(i+1)
    time.sleep(0.5)
    end = time.time()
    print("Loading time: {}".format(end - start))
    
    
    print(img.shape, kp.shape, hm.shape)

In [ ]:
start = time.time()
pool = Pool(4)
pool.apply()
end = time.time()
print("Loading time: {}".format(end - start))

In [ ]:
x,y,hm = train_generator_async.batches[0]

In [ ]:
len(train_generator_async.batches)

In [ ]:
# out = train_generator_async.batches[0].get()

In [ ]:
start = time.time()
for i in range(100):
    output = train_generator_async[i]
end = time.time()
print("Loading time: {}".format((end - start)/5.0))

In [ ]:
train_generator_async.batches[0]

In [ ]:
# train_generator = DataGenerator(train_annotations, FLAGS)
generator = data_generator(train_annotations, FLAGS)

In [ ]:
xbatch, ybatch, batch_gt_heatmap_np = next(generator)

Tf data experiment

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
import tensorflow as tf
import cv2 
import glob
import matplotlib.pyplot as plt

In [ ]:
paths = glob.glob('/root/data/aquabyte-images/erko-rotoy/2018-08-30/1/*')

In [ ]:
# # stupid generator
# def generator():
#     for path in paths:
#         yield cv2.imread(path)

In [ ]:
dataset = tf.data.Dataset.from_generator(data_generator,
                                         output_types=tf.int64).batch(4)

In [ ]:
training_dataset = tf.data.Dataset.from_generator(lambda: data_generator(train_annotations, FLAGS), 
                                                  output_types=(tf.int64, tf.int64, tf.float64))
iter = training_dataset.make_initializable_iterator()
el = iter.get_next()

In [ ]:
# training_dataset = tf.data.Dataset.from_generator(lambda: raw_data_gen(train_val_or_test=1), (tf.float32, tf.uint8), ([None, 1], [None]))


In [ ]:
# iter = dataset.make_initializable_iterator()
# el = iter.get_next()

In [ ]:
# with tf.Session() as sess:
#     sess.run(iter.initializer)
#     image = sess.run(el)
#     plt.imshow(image)
#     plt.show()
#     image = sess.run(el)
#     plt.imshow(image)
#     plt.show()
#     image = sess.run(el)
#     plt.imshow(image)
#     plt.show()

Using this:

https://github.com/cs230-stanford/cs230-code-examples/blob/master/tensorflow/vision/model/input_fn.py

1st option

In [ ]:
import time

In [ ]:
with tf.device('/cpu:0'):
    training_dataset = tf.data.Dataset.from_generator(lambda: data_generator(train_annotations, FLAGS), 
                                                      output_types=(tf.int64, tf.int64, tf.float64))
    iter = training_dataset.make_initializable_iterator()
    el = iter.get_next()

In [ ]:
with tf.Session() as sess:
    start = time.time()
    sess.run(iter.initializer)
    xbatch, ybatch, batch_gt_heatmap_np = sess.run(el)
    end = time.time()
    print(end-start)

Seems like this is better https://stackoverflow.com/questions/47086599/parallelising-tf-data-dataset-from-generator

In [ ]:
from utils.utils import load_image_keypoints
from utils.cpm_utils import make_heatmaps_from_joints

In [ ]:
num_parallel_calls = 4

In [ ]:
def new_gen():
    i = 0
    nsteps = 1000
    while True:
        i += 1 
        if i >= nsteps:
            i = 0
        yield train_annotations[i]

In [ ]:
# open_fn = lambda ann:
load_fn = lambda ann: load_image_keypoints(ann, FLAGS)
expand_fn = lambda x,y : (x, np.expand_dims(y, axis=0))
heatmap_fn = lambda x, y: (x, y, make_heatmaps_from_joints(FLAGS.input_size[0],
                                                           FLAGS.heatmap_size,
                                                           FLAGS.joint_gaussian_variance,
                                                           y))

In [ ]:
x,y = load_fn(train_annotations[0])
x, y  = expand_fn(x,y)
xa, ya, ga = heatmap_fn(x,y)

In [ ]:
# with tf.device('/cpu:0'):
#     dataset = tf.data.Dataset.from_generator(new_gen, tf.string)
# #     dataset = dataset.map(load_fn, num_parallel_calls=num_parallel_calls)
# #     dataset = dataset.map(expand_fn, num_parallel_calls=num_parallel_calls)
# #     dataset = dataset.map(heatmap_fn, num_parallel_calls=num_parallel_calls)
    
#     iter = dataset.make_initializable_iterator()
#     el = iter.get_next()

In [ ]:
# with tf.Session() as sess:
# #     start = time.time()
#     sess.run(iter.initializer)
#     print(sess.run(el))

In [ ]:
dataset = tf.data.Dataset.from_generator(new_gen, tf.string)

In [ ]:
with tf.device('/cpu:0'):
    dataset = (tf.data.Dataset.from_generator(new_gen, tf.string)
        .shuffle(buffer_size=1)  # whole dataset into the buffer ensures good shuffling
        .map(load_fn, num_parallel_calls=num_parallel_calls)
        .map(expand_fn, num_parallel_calls=num_parallel_calls)
        .map(heatmap_fn, num_parallel_calls=num_parallel_calls)
        .batch(4)
        .prefetch(1)  # make sure you always have one batch ready to serve
    )